In [ ]:
"""
Processed data from the WIDER FACE dataset by isolating bounding boxes using Google Vision.
"""
from PIL import Image
from google.cloud import vision
import os
import io
#credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/work/hackalytics-2022-edddba28c1d0.json'

outDir = "/work/processedFaces"

def detectFaces(imagePath)->list:
    """Detects faces in an image. 
    Takes in a bytes object representing an image.
    Returns a list of tuples containing two corners 
    defining the bounding box of each image.
    """
    outList = []
    client = vision.ImageAnnotatorClient()
    with io.open(imagePath, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    response = client.face_detection(image=image)
    faces = response.face_annotations

    for face in faces:
        vertices = ([(vertex.x, vertex.y) for vertex in face.bounding_poly.vertices])
        outList.append((vertices[1], vertices[3]))
    return outList

def processImage(imagePath, outDir):
    """
    Given an image's filepath, return a series of images cropped to facial bounding boxes to outDir.
    """
    im = Image.open(imagePath)
    imName = imagePath.split('/')[-1]
    for idx, bBox in enumerate(detectFaces(imagePath)):
        topRight = bBox[0]
        bottomLeft = bBox[1]
        im1 = im.crop((bottomLeft[0], topRight[1], topRight[0], bottomLeft[1]))
        if os.path.exists(outDir) == False:
            os.makedirs(outDir)
        im1.save(outDir + '/' + str(idx) + '-' + imName)
        



In [ ]:
outDir = "/work/new_images"

for subDir in os.listdir('/work/images'):
    for images in os.listdir('/work/images/' + subDir):
        processImage('/work/images/' + subDir + '/' + images, outDir + '/' + subDir)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d80e1763-e9c4-41f1-9d3f-7a71b565f358' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>